# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /private/var/folders/xv/6xdsnhm10jg5y1g0bm3kp9140000gp/T/pip-req-build-8g5oafk2
  Running command git clone -q git://github.com/Theano/Theano.git /private/var/folders/xv/6xdsnhm10jg5y1g0bm3kp9140000gp/T/pip-req-build-8g5oafk2
  Created wheel for Theano: filename=Theano-1.0.4+21.g8f510a1f0-cp36-none-any.whl size=2666379 sha256=041cb3a9e761cab5d820669c49be2cb71215f819b38d3ca8810aee93f4551093
  Stored in directory: /private/var/folders/xv/6xdsnhm10jg5y1g0bm3kp9140000gp/T/pip-ephem-wheel-cache-pri53nb7/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.4+21.g8f510a1f0
    Uninstalling Theano-1.0.4+21.g8f510a1f0:
      Successfully uninstalled Theano-1.0.4+21.g8f510a1f0
  Using cached https://files.pythonhosted.org/packages/27/8c/7608ba709bd536bc2bccb0d1abbb70aafe9cf7e0170353b4b720ed54cb71/tensorflow-1.14.0-cp36-cp36m-macosx_10_11_x86_64.whl
  Using cached http

# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/home/gabrielle/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/gabrielle/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [8]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 97us/step - loss: 0.4971 - accuracy: 0.8005
Epoch 2/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.4027 - accuracy: 0.8095
Epoch 3/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.3916 - accuracy: 0.8189
Epoch 4/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.3907 - accuracy: 0.8285
Epoch 5/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.4040 - accuracy: 0.8339
Epoch 6/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.3864 - accuracy: 0.8389
Epoch 7/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.3780 - accuracy: 0.8397
Epoch 8/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.3797 - accuracy: 0.8411
Epoch 9/100
8000/8000 [==============================] - 1s 89us/step - loss: 0.3814 - accuracy: 0.8420
Epoch 10/100
8000/8000 [==============================] - 1s 83u

8000/8000 [==============================] - 1s 81us/step - loss: 0.3500 - accuracy: 0.8520
Epoch 79/100
8000/8000 [==============================] - 1s 81us/step - loss: 0.3482 - accuracy: 0.8529
Epoch 80/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3474 - accuracy: 0.8524 0s - loss: 0.3479 - accuracy: 0.
Epoch 81/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.3512 - accuracy: 0.8514
Epoch 82/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.3566 - accuracy: 0.8530
Epoch 83/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.3492 - accuracy: 0.8535
Epoch 84/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.3479 - accuracy: 0.8537
Epoch 85/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3473 - accuracy: 0.8535
Epoch 86/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.3514 - accuracy: 0.8535 0s - loss: 0.3527 - accuracy
Epoch 

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [9]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [10]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1537   58]
 [ 210  195]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [11]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 91us/step - loss: 0.5112 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.4466 - accuracy: 0.7974
Epoch 3/100
8000/8000 [==============================] - 1s 85us/step - loss: 0.4314 - accuracy: 0.8056
Epoch 4/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4295 - accuracy: 0.8050
Epoch 5/100
8000/8000 [==============================] - 1s 93us/step - loss: 0.4289 - accuracy: 0.8050
Epoch 6/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4285 - accuracy: 0.8055
Epoch 7/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4283 - accuracy: 0.8035
Epoch 8/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4272 - accuracy: 0.8079
Epoch 9/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.4256 - accuracy: 0.8100
Epoch 10/100
8000/8000 [==============================] - 1s 88u

8000/8000 [==============================] - 1s 83us/step - loss: 0.3374 - accuracy: 0.8608
Epoch 79/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.3368 - accuracy: 0.8624
Epoch 80/100
8000/8000 [==============================] - 1s 97us/step - loss: 0.3368 - accuracy: 0.8611
Epoch 81/100
8000/8000 [==============================] - 1s 92us/step - loss: 0.3369 - accuracy: 0.8610
Epoch 82/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.3368 - accuracy: 0.8601
Epoch 83/100
8000/8000 [==============================] - 1s 98us/step - loss: 0.3364 - accuracy: 0.8625
Epoch 84/100
8000/8000 [==============================] - 1s 71us/step - loss: 0.3360 - accuracy: 0.8616
Epoch 85/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.3364 - accuracy: 0.8614
Epoch 86/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.3360 - accuracy: 0.8621
Epoch 87/100
8000/8000 [==============================] - 1s 79us/st

In [12]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1527   68]
 [ 206  199]]


# Initialising the 3rd ANN

In [14]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 98us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 88us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 86us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 87us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 97us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 101us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 107us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 97us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 86us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================] - 1s 9

Epoch 79/100
8000/8000 [==============================] - 1s 80us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 80/100
8000/8000 [==============================] - 1s 80us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 81/100
8000/8000 [==============================] - 1s 81us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 82/100
8000/8000 [==============================] - 1s 84us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 83/100
8000/8000 [==============================] - 1s 82us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 84/100
8000/8000 [==============================] - 1s 85us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 85/100
8000/8000 [==============================] - 1s 82us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 86/100
8000/8000 [==============================] - 1s 83us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 87/100
8000/8000 [==============================] - 1s 103us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 88/100
8000/8000 [==============================

In [15]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:{}\nmean:{}\nvariance:{}".format(accuracies, mean, variance))

# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [18]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [19]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [20]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/50
7200/7200 [==============================] - 1s 108us/step - loss: 0.4993 - accuracy: 0.7969
Epoch 2/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.4344 - accuracy: 0.7971
Epoch 3/50
7200/7200 [==============================] - 1s 90us/step - loss: 0.4336 - accuracy: 0.7971
Epoch 4/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.4320 - accuracy: 0.7971
Epoch 5/50
7200/7200 [==============================] - 1s 92us/step - loss: 0.4322 - accuracy: 0.7971
Epoch 6/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.4321 - accuracy: 0.7971
Epoch 7/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.4297 - accuracy: 0.7971
Epoch 8/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.4299 - accuracy: 0.7971
Epoch 9/50
7200/7200 [==============================] - 1s 92us/step - loss: 0.4262 - accuracy: 0.8142
Epoch 10/50
7200/7200 [==============================] - 1s 109us/step -

7200/7200 [==============================] - 1s 95us/step - loss: 0.4231 - accuracy: 0.8301
Epoch 30/50
7200/7200 [==============================] - 1s 99us/step - loss: 0.4213 - accuracy: 0.8308
Epoch 31/50
7200/7200 [==============================] - 1s 112us/step - loss: 0.4202 - accuracy: 0.8319
Epoch 32/50
7200/7200 [==============================] - 1s 100us/step - loss: 0.4198 - accuracy: 0.8328
Epoch 33/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.4232 - accuracy: 0.8328
Epoch 34/50
7200/7200 [==============================] - 1s 84us/step - loss: 0.4232 - accuracy: 0.8328
Epoch 35/50
7200/7200 [==============================] - 1s 86us/step - loss: 0.4214 - accuracy: 0.8319
Epoch 36/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4212 - accuracy: 0.8318
Epoch 37/50
7200/7200 [==============================] - 1s 97us/step - loss: 0.4217 - accuracy: 0.8308
Epoch 38/50
7200/7200 [==============================] - 1s 88us/step - lo

7200/7200 [==============================] - 1s 86us/step - loss: 0.4252 - accuracy: 0.8263
Epoch 8/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4216 - accuracy: 0.8279
Epoch 9/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4220 - accuracy: 0.8278
Epoch 10/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.4201 - accuracy: 0.8301
Epoch 11/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.4212 - accuracy: 0.8313
Epoch 12/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4224 - accuracy: 0.8304
Epoch 13/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.4240 - accuracy: 0.8314
Epoch 14/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.4175 - accuracy: 0.8304
Epoch 15/50
7200/7200 [==============================] - 1s 83us/step - loss: 0.4209 - accuracy: 0.8300
Epoch 16/50
7200/7200 [==============================] - 1s 85us/step - loss:

7200/7200 [==============================] - 1s 100us/step - loss: 0.3968 - accuracy: 0.8357
Epoch 36/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.3931 - accuracy: 0.8386
Epoch 37/50
7200/7200 [==============================] - 1s 100us/step - loss: 0.3961 - accuracy: 0.8357
Epoch 38/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.3942 - accuracy: 0.8383
Epoch 39/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.3952 - accuracy: 0.8367
Epoch 40/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3984 - accuracy: 0.8347
Epoch 41/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.3979 - accuracy: 0.8367
Epoch 42/50
7200/7200 [==============================] - 1s 105us/step - loss: 0.3941 - accuracy: 0.8385
Epoch 43/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.3937 - accuracy: 0.8389
Epoch 44/50
7200/7200 [==============================] - 1s 94us/step 

7200/7200 [==============================] - 1s 90us/step - loss: 0.4261 - accuracy: 0.8253
Epoch 14/50
7200/7200 [==============================] - 1s 86us/step - loss: 0.4236 - accuracy: 0.8253
Epoch 15/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4263 - accuracy: 0.8249
Epoch 16/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4234 - accuracy: 0.8296
Epoch 17/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.4275 - accuracy: 0.8265
Epoch 18/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4268 - accuracy: 0.8282
Epoch 19/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4241 - accuracy: 0.8267
Epoch 20/50
7200/7200 [==============================] - 1s 97us/step - loss: 0.4251 - accuracy: 0.8269
Epoch 21/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4264 - accuracy: 0.8263
Epoch 22/50
7200/7200 [==============================] - 1s 91us/step - los

KeyboardInterrupt: 

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)